## 3.3 Implémentation des relations n:n

### Exemple sous SQLAlchemy

In [2]:
from sqlalchemy import Table, Column,  ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker,  relationship

# Moteur d'accès à la base sqlalchemy.sqlite
engine = create_engine('sqlite:///sqlalchemy.sqlite')

<img src="../Relation_n_n.png">

In [1]:
import random
import string

# Génération d'une chaîne de caractères aléatoire
def get_random_string(length):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

<p id="declarations" style="font-size:120%">Déclaration des classes métier :</p>

In [19]:
Base = declarative_base()
Base.metadata.bind = engine

# Table intermédiaire
user_role = Table(
    "user_role",
    Base.metadata,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user.id")),
    Column("role_id", ForeignKey("role.id")),
)

# Définition des classes
class User(Base):
    __tablename__ = 'user'

    id = Column(Integer, primary_key=True)
    pseudo = Column(String(16), nullable=False)
    email = Column(String(255), nullable=False)
    pwd = Column(String(32), nullable=False)
    
    roles = relationship("Role", secondary=user_role, back_populates="users")
    
class Role(Base):
    __tablename__ = 'role'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(16), nullable=False)
    description = Column(String(255), nullable=False)
    
    users = relationship("User", secondary=user_role, back_populates="roles")
    

<p id="usage" style="font-size:120%">Usage :</p>

In [20]:
# Session
DBSession = sessionmaker(bind=engine)
session = DBSession()

In [21]:
# Création de rôles
roles = session.query(Role).all()
names = [role.name for role in roles]
if not 'admin' in names:
    admin = Role(name="admin",description="Administrateur de l'application")
    session.add(admin)
if not 'root' in names:
    root = Role(name="root",description="Root a tous les droits")
    session.add(root)
session.commit()

In [22]:
# Récupération d'un utilisateur
user = session.query(User).filter(User.pseudo == 'rde').one()
print(user.roles)

In [23]:
# Affectation d'un rôle à un utilisateur
admin = session.query(Role).filter(Role.name == "admin").one()
user.roles.append(admin)

session.add(user)
session.commit()

In [24]:
# La relation a été enregistrée dans la base, via la table intermédiaire
user = session.query(User).filter(User.pseudo == 'rde').one()
print([role.name for role in user.roles])

admin = session.query(Role).filter(Role.name == "admin").one()
print([user.pseudo for user in admin.users])

['admin']
['ijt', 'rde', 'upc']


In [25]:
# Création d'un utilisateur
pseudo = get_random_string(3)
user = User(pseudo=pseudo, email='{}@gmail.com'.format(pseudo), pwd=get_random_string(8))

# Ajout d'un utilisateur dans la liste d'un rôle
admin.users.append(user)

print(user.pseudo)

session.add(admin)
session.commit()

nym


In [26]:
# La relation a été enregistrée dans la base, via la table intermédiaire
check = session.query(User).filter(User.pseudo == user.pseudo).one()
print([role.name for role in check.roles])

admin = session.query(Role).filter(Role.name == "admin").one()
print([user.pseudo for user in admin.users])

['admin']
['ijt', 'rde', 'upc', 'nym']


In [27]:
# Suppression de l'utilisateur parmi les admin
admin.users.remove(check)
session.add(admin)
session.commit()

In [28]:
admin = session.query(Role).filter(Role.name == "admin").one()
print([user.pseudo for user in admin.users])

['ijt', 'rde', 'upc']
